In [75]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def send_request(url:str) -> str:
    response = requests.get(url)
    return response.text

def parse_drug_page(page_index:str) -> pd.DataFrame:
    
    soup = BeautifulSoup(page_index, 'html.parser')
    table_list = soup.find_all('ul', {'class': 'ddc-list-column-2'})

    data = []
    base_url = "https://www.drugs.com"
    for ul in table_list:
        for li in ul.find_all('li'):
            a_tag = li.find('a')
            medicine_name = a_tag.text
            medicine_link = a_tag['href']
            data.append([medicine_name, base_url + medicine_link])

    df = pd.DataFrame(data, columns=['Medicine name', 'Medicine Link'])

    return df

def get_drug_from_letter(url:str,letter:str) -> pd.DataFrame:
    url = url + letter + ".html"

    page_index = send_request(url)
    df = parse_drug_page(page_index)
    
    return df

In [76]:
def get_all_drugs(url:str) -> pd.DataFrame:
    all_drugs = pd.DataFrame()
    for i in range(97, 123):
        letter = chr(i)
        all_drugs = pd.concat([all_drugs, get_drug_from_letter(url, letter)])
    
    return all_drugs

In [ ]:
get_drug_from_letter(url = "https://www.drugs.com/alpha/",letter = 'b')

In [78]:
get_all_drugs(url = "https://www.drugs.com/alpha/")

,Medicine name,Medicine Link
0,Abilify,https://www.drugs.com/abilify.html
1,Abilify Asimtufii,https://www.drugs.com/abilify-asimtufii.html
2,Abilify Maintena,https://www.drugs.com/abilify-maintena.html
3,Abiraterone,https://www.drugs.com/abiraterone.html
4,Acetaminophen,https://www.drugs.com/acetaminophen.html
...,...,...
40,Zyloprim,https://www.drugs.com/cons/zyloprim.html
41,Zyprexa,https://www.drugs.com/zyprexa.html
42,Zyrtec,https://www.drugs.com/zyrtec.html
43,Zytiga,https://www.drugs.com/zytiga.html


In [84]:
import requests

# Define the base URL for the RxNorm API
base_url = "https://rxnav.nlm.nih.gov/REST/drugs.json"

# Define the parameters (searching for "cymbalta")
params = {
    "name": "cymbalta"
}

# Send the GET request
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print(data)  # Print the response in JSON format
else:
    print(f"Request failed with status code {response.status_code}")


{'drugGroup': {'name': None, 'conceptGroup': [{'tty': 'BPCK'}, {'tty': 'SBD', 'conceptProperties': [{'rxcui': '596928', 'name': 'duloxetine 20 MG Delayed Release Oral Capsule [Cymbalta]', 'synonym': 'Cymbalta 20 MG Delayed Release Oral Capsule', 'tty': 'SBD', 'language': 'ENG', 'suppress': 'N', 'umlscui': ''}, {'rxcui': '596932', 'name': 'duloxetine 30 MG Delayed Release Oral Capsule [Cymbalta]', 'synonym': 'Cymbalta 30 MG Delayed Release Oral Capsule', 'tty': 'SBD', 'language': 'ENG', 'suppress': 'N', 'umlscui': ''}, {'rxcui': '615186', 'name': 'duloxetine 60 MG Delayed Release Oral Capsule [Cymbalta]', 'synonym': 'Cymbalta 60 MG (as duloxetine HCl 67.3 MG) Delayed Release Oral Capsule', 'tty': 'SBD', 'language': 'ENG', 'suppress': 'N', 'umlscui': ''}]}]}}


Sending Request to OpenFDA API Endpoint

In [87]:
import requests

# Define the OpenFDA API endpoint for drug labeling
url = "https://api.fda.gov/drug/label.json"

# Parameters for searching by brand name (e.g., "Aspirin")
params = {
    'search': 'openfda.brand_name:"Abilify"',  # Replace "Aspirin" with any drug name
    'limit': 1  # Limit to 1 result
}

# Send the GET request to OpenFDA API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    if data.get('results'):
        drug_info = data['results'][0]
        
        # Print out some information about the drug
        print("Drug Name:", drug_info['openfda']['brand_name'][0])
        print("Indications and Usage:", drug_info.get('indications_and_usage', 'Not available'))
        print("Side Effects:", drug_info.get('adverse_reactions', 'Not available'))
        print("Dosage and Administration:", drug_info.get('dosage_and_administration', 'Not available'))
    else:
        print("No information found for the drug.")
else:
    print(f"Error retrieving data: {response.status_code}")


Drug Name: ABILIFY
Indications and Usage: ["1 INDICATIONS AND USAGE ABILIFY (aripiprazole) Tablets are indicated for the treatment of: Schizophrenia Acute Treatment of Manic and Mixed Episodes associated with Bipolar I Disorder Adjunctive Treatment of Major Depressive Disorder Irritability Associated with Autistic Disorder Treatment of Tourette's Disorder ABILIFY is an atypical antipsychotic. ABILIFY is indicated for: Schizophrenia ( 14.1 ) Acute Treatment of Manic and Mixed Episodes associated with Bipolar I Disorder ( 14.2 ) Adjunctive Treatment of Major Depressive Disorder ( 14.3 ) Irritability Associated with Autistic Disorder ( 14.4 ) Treatment of Tourette's Disorder ( 14.5 )"]
Side Effects: ["6 ADVERSE REACTIONS The following adverse reactions are discussed in more detail in other sections of the labeling: Increased Mortality in Elderly Patients with Dementia-Related Psychosis [see Boxed Warning and Warnings and Precautions (5.1) ] Cerebrovascular Adverse Events, Including Stroke

In [92]:
import pandas as pd
from fuzzywuzzy import process


df = pd.read_csv('all_drugs.csv')

def find_most_similar_medicine(user_input, df_column):
    best_match = process.extractOne(user_input, df_column)
    return best_match

user_input = "Amoxilin"  # Misspelled user input

most_similar = find_most_similar_medicine(user_input, df['Medicine name'])


# Print the result
print(f"User input: {user_input}")
print(f"Most similar medicine: {most_similar[0]} with a score of {most_similar[1]}")
print(f"Link: {df[df['Medicine name'] == most_similar[0]]['Medicine Link'].values[0]}")

User input: Amoxilin
Most similar medicine: Amoxicillin with a score of 84
Link: https://www.drugs.com/amoxicillin.html


In [114]:
import requests

# Base URL for the RxNorm API
BASE_URL = "https://rxnav.nlm.nih.gov/REST/approximateTerm.json"  # For JSON results

def get_drug_info(drug_name):
    # URL encoding the drug name to ensure it is safe for use in a URL
    term = drug_name.replace(" ", "%20")  # Manually replace spaces with URL encoded value if needed
    url = f"{BASE_URL}?term={term}&maxEntries=4&option=0"  # Option 0 for Current concepts
    
    # Make the request to the RxNorm API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Parse the JSON response
            return response.json()
        except ValueError as e:
            print(f"Error parsing JSON: {e}")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

# Example drug name to search for
drug_name = "Breztri Aerosphere"

# Call the function to get drug information
drug_info = get_drug_info(drug_name)

# If results are found, print the information
if drug_info:
    print(drug_info)
else:
    print("No drug information found or there was an error.")


{'approximateGroup': {'inputTerm': None, 'candidate': [{'rxcui': '2395467', 'rxaui': '12432310', 'score': '21.273706436157227', 'rank': '1', 'source': 'MMSL'}, {'rxcui': '2387325', 'rxaui': '12419262', 'score': '14.597760200500488', 'rank': '2', 'name': 'Breztri', 'source': 'RXNORM'}, {'rxcui': '2387324', 'rxaui': '12419265', 'score': '13.486606213569642', 'rank': '3', 'name': 'Breztri Inhalant Product', 'source': 'RXNORM'}, {'rxcui': '2387322', 'rxaui': '12432373', 'score': '13.438741250446913', 'rank': '4', 'source': 'MMSL'}]}}


In [115]:
drug_info

{'approximateGroup': {'inputTerm': None,
  'candidate': [{'rxcui': '2395467',
    'rxaui': '12432310',
    'score': '21.273706436157227',
    'rank': '1',
    'source': 'MMSL'},
   {'rxcui': '2387325',
    'rxaui': '12419262',
    'score': '14.597760200500488',
    'rank': '2',
    'name': 'Breztri',
    'source': 'RXNORM'},
   {'rxcui': '2387324',
    'rxaui': '12419265',
    'score': '13.486606213569642',
    'rank': '3',
    'name': 'Breztri Inhalant Product',
    'source': 'RXNORM'},
   {'rxcui': '2387322',
    'rxaui': '12432373',
    'score': '13.438741250446913',
    'rank': '4',
    'source': 'MMSL'}]}}